In [ ]:
import json
import re
import unicodedata

from bs4 import BeautifulSoup
from pathlib import Path

In [ ]:
INPUT_DIR = Path('../data/transcripts/')
OUTPUT_DIR = Path('../data/clean/')
TIMECODE = re.compile(r"^\d\d:\d\d:\d\d:\d\d\s*-\s*\d\d:\d\d:\d\d:\d\d$")

In [ ]:
for input_file in INPUT_DIR.glob("*.html"):
    with open(input_file, 'rt') as fh:
        raw_data = fh.read()
        soup = BeautifulSoup(raw_data, 'html.parser')

    parsed_document = []
    for paragraph in soup.find_all("p"):
        paragraph_text = unicodedata.normalize("NFKD", ' '.join(paragraph.text.strip().split())).strip()
        if paragraph_text == '':
            continue
        elif TIMECODE.match(paragraph_text):
            parsed_document.append({'timecode': paragraph_text})
        elif len(parsed_document) == 0:
            parsed_document.append({'timecode': 'N/A', 'text': paragraph_text})
        else:
            if 'text' not in parsed_document[-1]:
                parsed_document[-1]['text'] = paragraph_text
            else:
                parsed_document[-1] ['text'] += ' ' + paragraph_text

    with open(OUTPUT_DIR / input_file.name.replace('.html', '.json'), 'wt') as fh:
        json.dump(parsed_document, fh)

    with open(OUTPUT_DIR / input_file.name.replace('.html', '.markdown'), 'wt') as fh:
        for speaker in parsed_document:
            print(f"# {speaker['timecode']}\n{speaker['text']}\n", file=fh)